Data Augmentation : bert

# **setting**

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

#import os
#os.chdir('./data')
#os.getcwd()

In [2]:
def preprocessing_dataset(dataset):
    """처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    subject_entity = []
    object_entity = []

    for i, j in tqdm(zip(dataset["subject_entity"], dataset["object_entity"]), desc="preprocessing"):
        i = i[1:-1].split(",")[0].split(":")[1]
        j = j[1:-1].split(",")[0].split(":")[1]

        subject_entity.append(i)
        object_entity.append(j)

    out_dataset = pd.DataFrame(
        {
            "id": dataset["id"],
            "sentence": dataset["sentence"],
            "subject_entity": subject_entity,
            "object_entity": object_entity,
            "label": dataset["label"],
        }
    )
    return out_dataset


def load_data(dataset_dir):
    """csv 파일을 경로에 맡게 불러 옵니다."""
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_dataset(pd_dataset)

    return dataset

def label_to_num(label):
    num_label = []
    with open("/opt/ml/code/dict_label_to_num.pkl", "rb") as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label

**데이터 불러오기**

In [3]:
train_path = '/opt/ml/dataset/train/train.csv'
total_data = load_data(train_path)
# total_data.columns

preprocessing: 32470it [00:00, 376672.76it/s]


In [4]:
train_data = total_data.sample(frac=0.9, random_state=42)
val_data = total_data.drop(train_data.index)

train_label = label_to_num(train_data["label"].values)
val_label = label_to_num(val_data["label"].values)

In [5]:
pd.options.display.max_colwidth = 2000

# **Masked Language Model 활용 : klue/bert-base**   
- mlm활용해서 entity 바꾸고, 데이터 증강하기 : 실험

In [6]:
mlmtest = train_data.copy()
mlmtest = mlmtest.sort_values(by=["id"], ascending=[True])

### **01. MLM test**

In [7]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

2022-12-02 05:26:04.207654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
MODEL_NAME = 'klue/bert-base'
model = TFBertForMaskedLM.from_pretrained(MODEL_NAME, from_pt=True)   # from_pt=True pytorch로 학습된 모델을 텐서플로우에서 사용
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

2022-12-02 05:26:07.566648: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 05:26:07.567579: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initiali

In [9]:
from tqdm.auto import tqdm
def wordlist(x):
    return x['token_str'] if type(x) == dict else wordlist(x[0])
    
def findEntityAndChange(data,col,e1,e2):        # data : Series
    
    # step01
    mask_sent = data['sentence'].replace(e1,"[MASK]")           # sentence에 entity가 해당하는 부분 [MASK]로 바꾸기
    entities = list(map(wordlist,pip(mask_sent)))           # mlm 활용해서 대체할 eneity 리스트 뽑기 : 5개
    entities = entities[::-1]                                   # 순서 거꾸로 바꾸기(while문에서 pop() 사용하기 위해)
    
    while entities :
        entity = entities.pop()                                 # score 가장 높은 단어부터 확인하기
        if entity in ['[UNK]',')','(',',','#'] or '#' in entity or entity == e1 or entity == e2:     # sub,obj entity와 같거나 불용어 단어면 pass >> 불용어 처리할 단어 더 많을텐데 추가 수정사항에 해당
            entity =''
            continue
        else :
            break                                                  
    # step02
    change_sent = mask_sent.replace("[MASK]",entity)            # sentence 바꾸기
    
    # step03
    data['sentence'] = change_sent
    data[col] = '\''+entity+'\''
    return data                                   # Series 반환



def use_mlm(dataset):       #
    result_pd = pd.DataFrame()
    
    for idx, row in tqdm(dataset.iterrows(),desc="processig..."):
        #print('ind : ',row.id)
        sub = row['subject_entity'].strip()[1:-1]
        obj = row['object_entity'].strip()[1:-1]
    
        srow = findEntityAndChange(row.copy(),'subject_entity',sub,obj)
        orow = findEntityAndChange(row.copy(),'object_entity',obj,sub)
        
        # 빈값처리
        if srow.subject_entity !='':
            result_pd = result_pd.append(pd.Series(srow), ignore_index=True)
            
        if orow.object_entity !='':
            result_pd = result_pd.append(pd.Series(orow), ignore_index=True)
    
        
    return result_pd
    

In [10]:
#test5 = use_mlm(mlmtest)
#test5.to_csv('/opt/ml/workspace/save_entity_mt.csv',index=False,encoding = 'utf-8')

### **02.증강된 데이터와 원본 데이터 합치기**

In [11]:
def reformat(dataset):
    out_dataset = pd.DataFrame(
            {
                "id": dataset["id"],
                "sentence": dataset["sentence"],
                "subject_entity": "{'word' : "+dataset["subject_entity"]+", 'start_idx': 0, 'end_idx': 0, 'type': 'None'",
                "object_entity": "{'word' : "+dataset["object_entity"]+", 'start_idx': 0, 'end_idx': 0, 'type': 'None'",
                "label": dataset["label"],
            }
    )
    return out_dataset

In [12]:
# 01. 불러오기
en_pd = pd.read_csv('/opt/ml/dataset/train/aug_entities.csv')
#en_pd
train_path = '/opt/ml/dataset/train/train.csv'
train = pd.read_csv(train_path)

In [13]:
# 02. 합치기
re_en = reformat(en_pd)
en_data = pd.concat([train,re_en])

In [14]:
en_data

,id,sentence,subject_entity,object_entity,label,source
0,0.0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey Road》에 담은 노래다.,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26, 'type': 'ORG'}","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx': 18, 'type': 'PER'}",no_relation,wikipedia
1,1.0,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으로 재탄생한다.,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': 23, 'type': 'ORG'}","{'word': '대안신당', 'start_idx': 14, 'end_idx': 17, 'type': 'ORG'}",no_relation,wikitree
2,2.0,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터 관중 유치 성과와 마케팅 성과를 인정받아 ‘풀 스타디움상’과 ‘플러스 스타디움상’을 수상했다.,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 24, 'type': 'ORG'}","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx': 41, 'type': 'ORG'}",org:member_of,wikitree
3,3.0,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪고 있는 대구광역시에 행복박스를 전달했다고 10일 밝혔다.,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': 17, 'type': 'ORG'}","{'word': '박정부', 'start_idx': 22, 'end_idx': 24, 'type': 'PER'}",org:top_members/employees,wikitree
4,4.0,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8번으로 배정되었다.,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_idx': 30, 'type': 'ORG'}","{'word': '1967', 'start_idx': 0, 'end_idx': 3, 'type': 'DAT'}",no_relation,wikipedia
...,...,...,...,...,...,...
90911,32465.0,"한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등록 마감 결과, 강석호(3선·경북 영양·영덕·봉화·울진)-이장우(재선·대전 동구), 유기준(4선·비례)-박성중(초선·서울 서초을), 김선동(재선·서울 도봉을)-김종석(초선·비례), 심재철(5선·경기 안양시동안구을)-김재원(3선·경북 상주·군위·의성·청송) 등 4개 조가 등록했다고 밝혔다.","{'word' : {'word' : '유기준', 'start_idx': 0, 'end_idx': 0, 'type': 'None', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : {'word' : '비례', 'start_idx': 0, 'end_idx': 0, 'type': 'None', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",per:employee_of,NaN
90912,32467.0,신안군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 229개 기초지자체 중 최상위 등급인 A등급을 받으면서 전국에서 가장 높은 교통안전 인식 수준을 갖고 있는 것으로 나타났다.,"{'word' : {'word' : '신안군', 'start_idx': 0, 'end_idx': 0, 'type': 'None', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : {'word' : '신우철', 'start_idx': 0, 'end_idx': 0, 'type': 'None', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",org:top_members/employees,NaN
90913,32467.0,완도군(군수 신우)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 229개 기초지자체 중 최상위 등급인 A등급을 받으면서 전국에서 가장 높은 교통안전 인식 수준을 갖고 있는 것으로 나타났다.,"{'word' : {'word' : '완도군', 'start_idx': 0, 'end_idx': 0, 'type': 'None', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : {'word' : '신우', 'start_idx': 0, 'end_idx': 0, 'type': 'None', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",org:top_members/employees,NaN
90914,32469.0,"칠곡군(군수 구충곤)은 17일 동면의 이장 20여 명이 코로나 19 예방을 위해 버스 승강장, 공중화장실 등 다중이용시설과 오동리 천운아파트 주변을 소독하는 방역 봉사활동을 펼쳤다고 밝혔다.","{'word' : {'word' : '칠곡군', 'start_idx': 0, 'end_idx': 0, 'type': 'None', 'start_idx': 0, 'end_idx': 0, 'type': 'None'","{'word' : {'word' : '구충곤', 'start_idx': 0, 'end_idx': 0, 'type': 'None', 'start_idx': 0, 'end_idx': 0, 'type': 'None'",org:top_members/employees,NaN


In [15]:
#en_data.to_csv('/opt/ml/dataset/train/aug_entities.csv',index=False,encoding = 'utf-8')

---

# **Masked Language Model 실습**   

- klue/bert-base : 한국어 BERT

In [16]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

## **1.MLM,Tokenizer**   


- TFBertForMaskedLM.from_pretrained('BERT 모델 이름')을 넣으면 [MASK]라고 되어있는 단어를 맞추기 위한 마스크드 언어 모델링을 위한 구조로 BERT를 로드.
- AutoTokenizer.from_pretrained('모델 이름') : 해당 모델이 학습되었을 당시에 사용되었던 토크나이저를 로드.

In [17]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)   # from_pt=True pytorch로 학습된 모델을 텐서플로우에서 사용
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


## **2.BERT 입력**   


In [18]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')
inputs

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[   2, 4713, 2259, 3944, 6001, 2259,    4,  809,   18,    3]],
      dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

### **input_ids**
- klue/bert-base의 토크나이저를 사용하여 해당 문장을 정수 인코딩.
- 토크나이저로 변환된 결과에서 input_ids를 통해 정수 인코딩 결과를 확인

In [19]:
print(inputs['input_ids'])

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)


### **token_type_ids**
- 토크나이저로 변환된 결과에서 token_type_ids를 통해서 문장을 구분하는 세그먼트 인코딩 결과를 확인할 수 있습니다.

In [20]:
print(inputs['token_type_ids'])

tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)


### **attention_mask**
- 토크나이저로 변환된 결과에서 attention_mask를 통해서 실제 단어와 패딩 토큰을 구분하는 용도

In [21]:
print(inputs['attention_mask'])
# 현재의 입력에서는 패딩이 없으므로 여기서는 문장 길이만큼의 1 시퀀스를 얻음

tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


## **3. [MASK] 토큰 예측**   


In [22]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)   # from_pt=True pytorch로 학습된 모델을 텐서플로우에서 사용
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [23]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [24]:
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963515162467957,
  'token': 4559,
  'token_str': '스포츠',
  'sequence': '축구는 정말 재미있는 스포츠 다.'},
 {'score': 0.02595757506787777,
  'token': 568,
  'token_str': '거',
  'sequence': '축구는 정말 재미있는 거 다.'},
 {'score': 0.010033931583166122,
  'token': 3682,
  'token_str': '경기',
  'sequence': '축구는 정말 재미있는 경기 다.'},
 {'score': 0.007924363017082214,
  'token': 4713,
  'token_str': '축구',
  'sequence': '축구는 정말 재미있는 축구 다.'},
 {'score': 0.007844218984246254,
  'token': 5845,
  'token_str': '놀이',
  'sequence': '축구는 정말 재미있는 놀이 다.'}]

---

# **Masked Language Model 활용 : klue/bert-base**   

In [25]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [26]:
MODEL_NAME = 'klue/bert-base'
model = TFBertForMaskedLM.from_pretrained(MODEL_NAME, from_pt=True)   # from_pt=True pytorch로 학습된 모델을 텐서플로우에서 사용
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [27]:
text = '1년 후, 바이에른은 전설적인 오스트리아인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다.'
s = '에른스트 하펠'
o = '오스트리아'
text_s = text.replace(s,"[MASK]")
text_o = text.replace(o,"[MASK]")

In [28]:
text_s

'1년 후, 바이에른은 전설적인 오스트리아인 감독 [MASK]이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다.'

In [29]:
pip(text_s)

[{'score': 0.4391782581806183,
  'token': 1,
  'token_str': '[UNK]',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다.'},
 {'score': 0.07832354307174683,
  'token': 19920,
  'token_str': '케빈',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 케빈 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다.'},
 {'score': 0.02716059237718582,
  'token': 12603,
  'token_str': '필립',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 필립 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다.'},
 {'score': 0.02669833041727543,
  'token': 14051,
  'token_str': '마틴',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 마틴 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다.'},
 {'score': 0.019449416548013687,
  'token': 13,
  'token_str': ')',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 ) 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다.'}]

In [30]:
# train_data.loc[:10]

---

# **Masked Language Model 활용 : XLM-RoBERTa**   

In [31]:
# XLM-RoBERTa 토크나이저를 불러오기: https://huggingface.co/xlm-roberta-large
from transformers import XLMRobertaTokenizer,TFBertForMaskedLM,AutoTokenizer

In [ ]:
MODEL_NAME = 'xlm-roberta-large'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFBertForMaskedLM.from_pretrained(MODEL_NAME, from_pt=True)   # from_pt=True pytorch로 학습된 모델을 텐서플로우에서 사용

In [33]:
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

In [34]:
text = '이날 행사에는 김승남지역위원장을 비롯 우원식(전)원내대표, 송갑석광주시당위원장, 김철우보성군수, 이승옥강진군수, 박병종(전)고흥군수, 김성(전)장흥군수와 4개지역 더불어민주당 소속 도의원 9명, 군의원 31명을 포함 당직자 및 당원 3,500여명이 참석하여 성황리에 개최됐다.'
s = '우원식'
o = '원내대표'
text_s = text.replace(s,"[MASK]")
text_o = text.replace(o,"[MASK]")

In [35]:
MASK_TOKEN = tokenizer.mask_token
pip("1년 후, 바이에른은 전설적인 {}인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다..".format(MASK_TOKEN))

[{'score': 4.2155752453254536e-05,
  'token': 111204,
  'token_str': '很棒',
  'sequence': '1년 후, 바이에른은 전설적인很棒 인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다..'},
 {'score': 3.8323556509567425e-05,
  'token': 132616,
  'token_str': 'pregleda',
  'sequence': '1년 후, 바이에른은 전설적인 pregleda 인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다..'},
 {'score': 3.7428286304930225e-05,
  'token': 81126,
  'token_str': 'vôbec',
  'sequence': '1년 후, 바이에른은 전설적인 vôbec 인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다..'},
 {'score': 3.650986036518589e-05,
  'token': 52216,
  'token_str': '游客',
  'sequence': '1년 후, 바이에른은 전설적인游客 인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다..'},
 {'score': 3.5745335480896756e-05,
  'token': 194305,
  'token_str': 'రైతుల',
  'sequence': '1년 후, 바이에른은 전설적인 రైతుల 인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다..'}]

# **Masked Language Model 활용 : klue/roberta-large**   

In [36]:
MODEL_NAME = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = TFBertForMaskedLM.from_pretrained(MODEL_NAME, from_pt=True)   # from_pt=True pytorch로 학습된 모델을 텐서플로우에서 사용
from transformers import FillMaskPipeline
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['roberta.encoder.layer.22.attention.output.dense.bias', 'roberta.encoder.layer.17.attention.output.dense.bias', 'roberta.encoder.layer.5.attention.output.dense.weight', 'roberta.encoder.layer.20.attention.output.LayerNorm.bias', 'roberta.encoder.layer.7.output.dense.weight', 'roberta.encoder.layer.5.attention.self.query.bias', 'roberta.encoder.layer.10.attention.output.dense.weight', 'roberta.encoder.layer.5.output.dense.bias', 'roberta.encoder.layer.11.attention.self.value.weight', 'roberta.encoder.layer.23.attention.self.key.bias', 'roberta.encoder.layer.22.output.dense.bias', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.16.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.bias', 'robe

In [37]:
MASK_TOKEN = tokenizer.mask_token
pip("1년 후, 바이에른은 전설적인 오스트리아인 감독 {}이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다..".format(MASK_TOKEN))

[{'score': 0.0002880006795749068,
  'token': 16479,
  'token_str': '동맥',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 동맥 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다..'},
 {'score': 0.00024647614918649197,
  'token': 5223,
  'token_str': '부상',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 부상 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다..'},
 {'score': 0.0002279666077811271,
  'token': 731,
  'token_str': '낳',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 낳 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다..'},
 {'score': 0.00022247455490287393,
  'token': 4796,
  'token_str': '건축',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 건축 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다..'},
 {'score': 0.00021462894801516086,
  'token': 8305,
  'token_str': '피고',
  'sequence': '1년 후, 바이에른은 전설적인 오스트리아인 감독 피고 이 이끄는 함부르크 SV와 DFB - 포칼 결승에서 만났다..'}]

---